In [3]:
pip install requests html5lib bs4 re

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install re

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


## Characters

In [15]:
### Workflow
import requests 
from bs4 import BeautifulSoup 
import re

## A) Scrape https://onepiece.fandom.com/wiki/List_of_Canon_Characters for list of links towards all canon characters

## B) Get rid of the fluff and save the html output for caching purposes (no overloading)

## C) Scrape page as json based on Content
### 1 Appearance
### 2 Personality
### 3 Abilities and Powers | Abilities | Power
### 4 History
### 5 References

## C) Scrape for the Statistics and Devil Fruit informations (pi-items)

#Test run with Alpaca

In [8]:
charac_name = "Alpacaman"
charac_url = f"https://onepiece.fandom.com/wiki/{charac_name}"

r = requests.get(charac_url) 
soup = BeautifulSoup(r.content, 'html5lib')

print(soup.div)
print(charac_url)
# Add some logging / wait time

<div class="notifications-placeholder">
				<div class="banner-notifications-placeholder">
			<div class="wds-banner-notification__container">
	
</div>
		</div>
	</div>
https://onepiece.fandom.com/wiki/Alpacaman


In [85]:
toc_pattern = re.compile(r'toclevel-1.*')
tocs = soup.find_all("li", class_=toc_pattern)

def toc_to_json(tocs):
    """
    Scrap table of content and returns a json.

    Args:
        tocs (str): html soup of table of contents, filtered through class patterns

    Returns:
        json
        
    Remarks:
        #find_all recursive=False to only consider direct children and avoid dup entries with nested children

    """
    
    toc_data = []
    for li in tocs:
        entry = {
            'name': li.find('span', class_='toctext').get_text(),
            'children': toc_to_json(li.find('ul').find_all('li', recursive=False) if li.find('ul') else []) #recursive bs
        }
        toc_data.append(entry)
    return toc_data

toc_data = toc_to_json(tocs)
toc_data
#tocs


[{'name': 'Appearance', 'children': []},
 {'name': 'Personality', 'children': []},
 {'name': 'Abilities and Powers',
  'children': [{'name': 'Devil Fruit',
    'children': [{'name': 'Anime-Only Techniques', 'children': []}]},
   {'name': 'Weapons', 'children': []},
   {'name': 'Haki', 'children': []}]},
 {'name': 'History',
  'children': [{'name': 'Wano Country Arc', 'children': []}]},
 {'name': 'Major Battles', 'children': []},
 {'name': 'Trivia', 'children': []},
 {'name': 'References', 'children': []},
 {'name': 'Site Navigation', 'children': []}]

In [134]:
import re

def clean_content(content):
    # Add a space before an opening square bracket if preceded by a word character
    content = re.sub(r'(\w)(\[)', r'\1 [', content)
    
    # Add a space after a closing square bracket if followed by a word character
    content = re.sub(r'(\])(\w)', r'] \2', content)
    
    # Add a space before an opening parenthesis if preceded by a word character
    content = re.sub(r'(\w)(\()', r'\1 (', content)
    
    # Add a space after a closing parenthesis if followed by a word character
    content = re.sub(r'(\))(\w)', r') \2', content)
    
    # Add a space before a non-breaking space if preceded by a word character
    content = re.sub(r'(\w)(\\xa0)', r'\1 \\xa0', content)
    
    # Add a space after a non-breaking space if followed by a word character
    content = re.sub(r'(\\xa0)(\w)', r'\\xa0 \2', content)
    
    # Add a space between Japanese and Latin characters
    content = re.sub(r'([\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff])([A-Za-z])', r'\1 \2', content)
    content = re.sub(r'([A-Za-z])([\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff])', r'\1 \2', content)
    
    # Add a space between Latin characters and Japanese punctuation
    content = re.sub(r'([\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff]),', r'\1 ,', content)
    content = re.sub(r',([\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff])', r', \1', content)
    
    # Add a space between a lowercase letter and a capitalized letter within camelCase words
    content = re.sub(r'([a-z])([A-Z])', r'\1 \2', content)
    
    # Ensure there is a space before and after a question mark if it is within a word
    content = re.sub(r'(\w)\?(\w)', r'\1 ? \2', content)
    
    # Add spaces before and after parentheses within words
    content = re.sub(r'(\w)\((\w)', r'\1 (\2', content)
    content = re.sub(r'(\w)\)(\w)', r'\1) \2', content)
    
    return content

# Example usage
content = "He calls this styleAlpaca Kenpo(アルパカ剣法,Arupaka Kenpō?, Viz/Funi: Alpaca Fencing).[4]"
cleaned_content = clean_content(content)
print(cleaned_content)

def extract_text_with_spacing(element):
    text = ""
    last_text = ""
    for child in element.descendants:
        if child.name == 'a':
            hyperlink_text = ' ' + child.get_text(strip=True) + ' '
            if last_text.strip() != hyperlink_text.strip():
                text += hyperlink_text
                last_text = hyperlink_text
        elif child.string:
            child_text = child.string
            if last_text.strip() != child_text.strip():
                text += child_text
                last_text = child_text
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

He calls this style Alpaca Kenpo (アルパカ剣法 ,Arupaka Kenpō?, Viz/Funi: Alpaca Fencing).[4]


In [147]:
def extract_text_with_spacing(element):
    text = ""
    last_text = ""
    
    for child in element.descendants:
        if child.name == 'a':
            # Extract text from hyperlink with surrounding spaces
            hyperlink_text = ' ' + child.get_text(strip=True) + ' '
            # Add hyperlink text if it's not a duplicate of the last added text
            if hyperlink_text.strip() != last_text.strip():
                text += hyperlink_text
                last_text = hyperlink_text
        elif child.string:
            # Extract text from non-hyperlink elements
            child_text = child.string
            # Add child text if it's not a duplicate of the last added text
            if child_text.strip() != last_text.strip():
                text += child_text
                last_text = child_text

    # Clean up extra spaces around punctuation and possessives
    text = re.sub(r'\s+([.,!?\'":;])', r'\1', text)  # Remove space before punctuation
    text = re.sub(r'([\'":;])\s+', r'\1 ', text)  # Ensure space after punctuation
    text = re.sub(r'(\w)\'s', r'\1\'s', text)  # Remove space before 's for possessives
    text = re.sub(r'\(\s+', '(', text)  # Remove space after opening parenthesis
    text = re.sub(r'\s+\)', ')', text)  # Remove space before closing parenthesis
    text = re.sub(r'\[\s+', '[', text)  # Remove space after opening square bracket
    text = re.sub(r'\s+\]', ']', text)  # Remove space before closing square bracket
    
    # Fix the issue with backslashes before apostrophes
    text = text.replace("\\'", "'")
    
    # Clean up extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def enrich_toc(toc_data, soup):
    content_data = []
    for item in toc_data: 
        
        #TODO: Skip Site navigation or clean after ? Implemented for now, but need to decide. Maybe better to skip if need to add tables in the future (site nav data is in tables)
        if item['name'].lower() == 'site navigation':
            continue
        
        content_id = item['name'].replace(" ", "_")
        heading = soup.find(id = content_id).parent
        content_text = ""
        
        def process_sibling(sibling):
            nonlocal content_text
            # Next section is a paragraph
            if sibling.name == 'p':
                content_text += extract_text_with_spacing(sibling) + " "
            # Next section is an unordered or ordered list
            elif sibling.name in ['ul', 'ol']:
                for li in sibling.find_all('li'):
                    content_text += extract_text_with_spacing(li) + " "
            # Edge case if next section is a div, access it then run the above
            elif sibling.name == 'div':
                for sub_sibling in sibling.children:
                    process_sibling(sub_sibling)
        
        for sibling in heading.find_next_siblings():
            # Next section is a new heading, i.e. current is finished or empty
            if sibling.name in ['h2', 'h3', 'h4', 'h5', 'h6']:
                break
            process_sibling(sibling)
            
        content_entry = {
            'name': item['name'],
            'content': content_text.strip(),
            'children': enrich_toc((item['children'] if item['children'] else []), soup) #recursive bs
        }
        
        content_data.append(content_entry)
            
    return content_data

enrich_toc(toc_data, soup)

[{'name': 'Appearance',
  'content': "Alpacaman is a tall and tan skinned muscular man whose neck and gray head are those of an alpaca's, increasing his height. He wears an open dark leather jacket with two chains and yellow cape with feather-like frills, as well as gloves that don't match which includes a black glove on the right hand and a brown glove with two metal hoops on the left, as well as a pair of leather briefs with pockets on them and also black shoes. He also wears a red and white vertically striped pant leg on only his left leg and also a calf armor, leaving his right one bare.[1]",
  'children': []},
 {'name': 'Personality',
  'content': 'Alpacaman is very loyal to his crew, as he looked forward to the punishments that Eustass Kid would receive upon being caught. He likes to abuse his position as a guard to spit on the prisoners, and is quick to remind them of the punishments for retaliating against him.[1] Like many other people, he has a unique laughter style:"Pacacaca

In [47]:
def enrich_toc_to_data()

{'name': 'Devil Fruit',
 'children': [{'name': 'Anime-Only Techniques', 'children': []}]}